## **Turi Create Recommender on Skafos**
The below example code and more documentation on the approach can be found [here](https://apple.github.io/turicreate/docs/userguide/recommender).
This notebook does the following:
1. Loads the data
2. Builds the model
3. Saves the model

In [ ]:
from common.load_turicreate import *
tc = install_turicreate(timeout = 500, retries = 2)
import pandas as pd
import time
from skafossdk import *
import common.save_models as sm

In [ ]:
ska = Skafos()

### 1. **Load the data**
The data for this example comes from the open source MovieLens dataset.Here we fetch training data from an AWS bucket publicly hosted by Metis Machine. The data is then split into training and testing sets.

In the example here, we have movies (items) and ratings by users on those movies(actions).

*Note: We used a subset of the whole dataset because it results in a smaller model.* 

In [ ]:
%%capture 
actions = tc.SFrame.read_csv('https://s3.amazonaws.com/skafos.example.data/MovieLensDataset/ml-20m/ratings.csv'); 
items = tc.SFrame.read_csv('https://s3.amazonaws.com/skafos.example.data/MovieLensDataset/ml-20m/movies.csv');

In [ ]:
pd_actions = actions.to_dataframe() # convert to pandas dataframe
sample_user_ids = pd.Series(pd_actions['userId'].unique()).sample(25000) # take a sample of users
actions = tc.SFrame(pd_actions[pd_actions['userId'].isin(sample_user_ids)]) # filter the dataframe

In [ ]:
# split the training and validation sets up
training_data, validation_data = tc.recommender.util.random_split_by_user(actions, 'userId', 'movieId')

### 2. **Build the model**
We pass the data to the `tc.recommender.create` function and specify a few arguments needed to properly run the model. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/userguide/recommender/).

In [ ]:
# build the recommender
model = tc.recommender.create(training_data, 'userId', 'movieId')

### 3. **Save the model**
We save the model to Skafos so that you can download it into your project later without having to always retrain the model.

In [ ]:
# export to coreml
coreml_model_name = "recommender.mlmodel"
res = model.export_coreml(coreml_model_name)

# compress the model
compressed_model_name, compressed_model = sm.compress_model(coreml_model_name)

# save to Skafos
sm.skafos_save_model(skafos = ska, model_name = compressed_model_name,
                     compressed_model = compressed_model,
                     permissions = 'public')